<a href="https://colab.research.google.com/github/yagoandres/dea_studies/blob/main/julia_malmquist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
using Pkg; Pkg.add("DataEnvelopmentAnalysis")
import Pkg; Pkg.add("XLSX")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
using DataEnvelopmentAnalysis
using XLSX
file_path = "/content/20250914_DEA_EnergySecurityPaper2.xlsx"
xf = XLSX.readxlsx(file_path)


XLSXFile("/content/20250914_DEA_EnergySecurityPaper2.xlsx") containing 10 Worksheets
            sheetname size          range        
-------------------------------------------------
                   IO 18x2          A1:B18       
I1-Installed Capacit… 48x25         A1:Y48       
  Calculations of HHI 828x40        A1:AN828     
I2-Energy diversific… 71x39         A1:AM71      
     I3-Energy import 48x39         A1:AM48      
        I4-%Renewable 58x39         A1:AM58      
        O1-MWh capita 21x26         A1:Z21       
        O2-GDP_capita 21x26         A1:Z21       
   O3-Electrification 21x26         A1:Z21       
  O4-carbon intensity 25x26         A1:Z25       


In [ ]:
# Get the list of sheet names
sheet_names = XLSX.sheetnames(xf)

println("Checking sheet names:")
i_count = 0
o_count = 0

for sheet_name in sheet_names
    # Convert sheet name to lowercase for case-insensitive comparison
    lower_sheet_name = lowercase(sheet_name)

    if startswith(lower_sheet_name, "i")
        println("$(sheet_name) starts with 'I'")
        i_count += 1
    elseif startswith(lower_sheet_name, "o")
        println("$(sheet_name) starts with 'O'")
        o_count += 1
    else
        println("'$(sheet_name)' does not start with 'I' or 'O'")
    end
end

println("\nNumber of sheets starting with 'I':  $i_count")
println("Number of sheets starting with 'O':   $o_count")

Checking sheet names:
IO starts with 'I'
I1-Installed Capacity Capita starts with 'I'
'Calculations of HHI' does not start with 'I' or 'O'
I2-Energy diversification HHI starts with 'I'
I3-Energy import starts with 'I'
I4-%Renewable starts with 'I'
O1-MWh capita starts with 'O'
O2-GDP_capita starts with 'O'
O3-Electrification starts with 'O'
O4-carbon intensity starts with 'O'

Number of sheets starting with 'I':  5
Number of sheets starting with 'O':   4


In [ ]:
X = Array{Float64,3}(undef, 20, i_count-1, 24);
Y = Array{Float64,3}(undef, 20, o_count, 24);



In [ ]:
sh=xf["I1-Installed Capacity Capita"]
input_rd_expenditure=sh["B2:Y21" ]


20×24 Matrix{Any}:
 0.72   0.745  0.743  0.741  0.738  …  0.966  1.012  1.034  1.033  1.046
 1.866  1.867  1.865  1.87   1.882     2.203  2.227  2.251  2.273  2.294
 0.393  0.396  0.397  0.406  0.42      0.607  0.62   0.634  0.647  0.661
 3.652  3.663  3.704  3.825  3.812     4.035  4.073  4.107  4.098  4.041
 0.258  0.271  0.284  0.309  0.346     1.439  1.577  1.7    1.832  2.091
 1.751  1.764  1.775  1.783  1.79   …  1.786  1.787  1.79   1.79   1.792
 1.375  1.392  1.406  1.418  1.436     2.633  2.672  2.698  2.727  2.76
 0.096  0.1    0.102  0.106  0.11      0.261  0.267  0.275  0.281  0.286
 0.108  0.11   0.112  0.114  0.117     0.179  0.184  0.189  0.194  0.199
 1.29   1.301  1.311  1.321  1.332     1.593  1.618  1.678  1.708  1.736
 1.789  1.801  1.814  1.826  1.842  …  2.126  2.149  2.174  2.2    2.227
 0.499  0.501  0.503  0.505  0.51      0.679  0.695  0.711  0.728  0.744
 1.403  1.422  1.482  1.497  1.514     1.794  1.808  1.817  1.826  1.836
 3.374  3.376  3.384  3.39   3.39

In [ ]:
# Check the dimensions of X and input_rd_expenditure
println("Dimensions of X: ", size(X))
println("Dimensions of input_rd_expenditure: ", size(input_rd_expenditure))
println(size(X,1))
println(size(input_rd_expenditure,1))
println(size(X,3))
println(size(input_rd_expenditure,2))
# Ensure the dimensions are compatible for assignment
if size(X, 1) == size(input_rd_expenditure, 1) # and size(X, 3) == size(input_rd_expenditure, 2)
    # Iterate through the third dimension of X and assign columns
    for k in 1:size(X, 3)
        X[:, 1, k] = input_rd_expenditure[:, k]
    end
    println("Successfully assigned columns to X.")
else
    println("Dimension mismatch: Cannot assign input_rd_expenditure to X.")
end

Dimensions of X: (20, 4, 24)
Dimensions of input_rd_expenditure: (20, 24)
20
20
24
24
Successfully assigned columns to X.


In [ ]:
println("Checking sheet names:")
i_count = 0
o_count = 0

for sheet_name in sheet_names
    # Convert sheet name to lowercase for case-insensitive comparison
    lower_sheet_name = lowercase(sheet_name)
    sh=xf[sheet_name]
    table=sh["B2:Y21" ]
    if startswith(sheet_name,"IO")
      continue
    end
    if startswith(lower_sheet_name, "i")
        println("$(sheet_name) starts with 'I'")

        if size(X, 1) == size(table, 1) # and size(X, 3) == size(input_rd_expenditure, 2)
             # Iterate through the third dimension of X and assign columns
              for k in 1:size(X, 3)
                X[:, i_count+1, k] = table[:, k]
              end
              println("Successfully assigned columns to X.")
        else
          println("Dimension mismatch: Cannot assign input_rd_expenditure to X.")
          println(size(X,1))
          println(size(table,1))
          println(size(X,3))
          println(size(table,2))
        end # Added missing 'end' for the inner if block
        i_count += 1
    elseif startswith(lower_sheet_name, "o")
        println("$(sheet_name) starts with 'O'")
        if size(Y, 1) == size(table, 1) # and size(X, 3) == size(input_rd_expenditure, 2)
             # Iterate through the third dimension of X and assign columns
              for k in 1:size(Y, 3)
                Y[:, o_count+1, k] = table[:, k]
              end
              println("Successfully assigned columns to Y.")
        else
          println("Dimension mismatch: Cannot assign input_rd_expenditure to Y.")
          println(size(Y,1))
          println(size(table,1))
          println(size(Y,3))
          println(size(table,2))
        end # Added missing 'end' for the inner if block


        o_count += 1
    else
        println("'$(sheet_name)' does not start with 'I' or 'O'")
    end
end

Checking sheet names:
I1-Installed Capacity Capita starts with 'I'
Successfully assigned columns to X.
'Calculations of HHI' does not start with 'I' or 'O'
I2-Energy diversification HHI starts with 'I'
Successfully assigned columns to X.
I3-Energy import starts with 'I'
Successfully assigned columns to X.
I4-%Renewable starts with 'I'
Successfully assigned columns to X.
O1-MWh capita starts with 'O'
Successfully assigned columns to Y.
O2-GDP_capita starts with 'O'
Successfully assigned columns to Y.
O3-Electrification starts with 'O'
Successfully assigned columns to Y.
O4-carbon intensity starts with 'O'
Successfully assigned columns to Y.


In [ ]:
println(X)

[0.72 4114.3151339999995 -33.8428334748467 11.557; 1.866 7029.982510000001 -117.602689924911 3.765; 0.393 7649.75491 23.6412025308101 40.939; 3.652 4047.171753 -50.614077254633 28.536; 0.258 6400.3657939999985 2.75465294883407 5.336; 1.751 6260.104388000001 52.6597179220674 6.409; 1.375 3710.5159100000005 61.1039580456091 2.738; 0.096 4975.500104000001 21.88108421401 6.042; 0.108 2626.1350079999993 -64.2403511903661 3.699; 1.29 2925.1148359999997 88.8679040891303 6.853; 1.789 2201.5143620000003 82.8158242809183 4.603; 0.499 3018.97641 -47.9631352840124 6.665; 1.403 2769.581916 -56.4420863609392 6.354; 3.374 5031.505922 -348.213745162547 0.0; 0.872 8700.491913999997 -29.8190312056668 0.386; 1.162 3205.611601 88.8626965698761 0.508; 0.439 3005.8575220000002 66.4118980253342 10.096; 1.307 3080.0505919999996 -18.0990443841151 1.087; 2.835 3392.8632410000005 26.6791454298185 3.856; 1.333 2419.386806 67.1355045030986 6.501;;; 0.745 4016.492525 -44.4658027559469 15.036; 1.867 6606.175154 -126

In [ ]:
println("Printing values in X:")
for j in 1:size(X, 2)
    for k in 1:size(X, 3)
        println("X[:, $(j), $(k)]:")
        println(X[:, j, k])
    end
end

Printing values in X:
X[:, 1, 1]:
[0.72, 1.866, 0.393, 3.652, 0.258, 1.751, 1.375, 0.096, 0.108, 1.29, 1.789, 0.499, 1.403, 3.374, 0.872, 1.162, 0.439, 1.307, 2.835, 1.333]
X[:, 1, 2]:
[0.745, 1.867, 0.396, 3.663, 0.271, 1.764, 1.392, 0.1, 0.11, 1.301, 1.801, 0.501, 1.422, 3.376, 0.872, 1.188, 0.448, 1.311, 2.859, 1.35]
X[:, 1, 3]:
[0.743, 1.865, 0.397, 3.704, 0.284, 1.775, 1.406, 0.102, 0.112, 1.311, 1.814, 0.503, 1.482, 3.384, 0.871, 1.217, 0.458, 1.314, 2.884, 1.366]
X[:, 1, 4]:
[0.741, 1.87, 0.406, 3.825, 0.309, 1.783, 1.418, 0.106, 0.114, 1.321, 1.826, 0.505, 1.497, 3.39, 0.871, 1.246, 0.474, 1.316, 2.928, 1.382]
X[:, 1, 5]:
[0.738, 1.882, 0.42, 3.812, 0.346, 1.79, 1.436, 0.11, 0.117, 1.332, 1.842, 0.51, 1.514, 3.392, 0.87, 1.273, 0.49, 1.317, 2.971, 1.397]
X[:, 1, 6]:
[0.732, 1.897, 0.43, 3.858, 0.398, 1.791, 1.454, 0.112, 0.12, 1.342, 1.858, 0.513, 1.533, 3.414, 0.87, 1.312, 0.514, 1.316, 3.012, 1.424]
X[:, 1, 7]:
[0.725, 1.916, 0.445, 3.866, 0.478, 1.788, 1.484, 0.113, 0.124, 1

In [ ]:
println("Printing values in Y:")
for j in 1:size(Y, 2)
    for k in 1:size(Y, 3)
        println("Y[:, $(j), $(k)]:")
        println(Y[:, j, k])
    end
end

Printing values in Y:
Y[:, 1, 1]:
[0.002396144682063065, 0.011333115831729862, 0.002005134760910594, 0.01954631520461317, 0.001067753748574963, 0.008953045131443802, 0.006955617062686743, 0.0005401056362270469, 0.00045372557288962494, 0.004713285190682977, 0.008656926779735811, 0.0020645773053757943, 0.005982644116046066, 0.008572921633643187, 0.004467164233033682, 0.006210627890737767, 0.0019010816968384543, 0.0063846491536572346, 0.013507335736306397, 0.006131923432217355]
Y[:, 1, 2]:
[0.0025829644656894806, 0.011580457768379793, 0.001863572600324556, 0.018830293898483074, 0.0011580206756103095, 0.009057661950695205, 0.007065945517281, 0.000543733927934016, 0.0004845323587697029, 0.004744728295376525, 0.008511230818711508, 0.0021001196844441803, 0.006099691197957465, 0.008572680652418025, 0.004416949894906656, 0.006587107384708988, 0.0018452691963811892, 0.006489840154142348, 0.013112338074591872, 0.0062973746685511065]
Y[:, 1, 3]:
[0.0026965529561946695, 0.011405282469179098, 0.0019

In [ ]:
malmquist(X, Y)

┌ Warning: DMU 9 termination status: INFEASIBLE. Primal status: NO_SOLUTION. Dual status: INFEASIBILITY_CERTIFICATE
└ @ DataEnvelopmentAnalysis ~/.julia/packages/DataEnvelopmentAnalysis/xHTKm/src/dea.jl:164
┌ Warning: DMU 14 termination status: INFEASIBLE. Primal status: NO_SOLUTION. Dual status: INFEASIBILITY_CERTIFICATE
└ @ DataEnvelopmentAnalysis ~/.julia/packages/DataEnvelopmentAnalysis/xHTKm/src/dea.jl:164
┌ Warning: DMU 9 termination status: INFEASIBLE. Primal status: NO_SOLUTION. Dual status: INFEASIBILITY_CERTIFICATE
└ @ DataEnvelopmentAnalysis ~/.julia/packages/DataEnvelopmentAnalysis/xHTKm/src/dea.jl:164
┌ Warning: DMU 9 termination status: INFEASIBLE. Primal status: NO_SOLUTION. Dual status: INFEASIBILITY_CERTIFICATE
└ @ DataEnvelopmentAnalysis ~/.julia/packages/DataEnvelopmentAnalysis/xHTKm/src/dea.jl:164
┌ Warning: DMU 14 termination status: INFEASIBLE. Primal status: NO_SOLUTION. Dual status: INFEASIBILITY_CERTIFICATE
└ @ DataEnvelopmentAnalysis ~/.julia/packages/DataEnve

ArgumentError: ArgumentError: colnms should have length 0 or 69